In [16]:
from troposphere import Template, Ref, Tags, Output, Parameter, FindInMap, GetAtt
from troposphere.autoscaling import AutoScalingGroup, LaunchConfiguration, Tag
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancingv2 as LoadBalancer

In [17]:
resources = set()
t = Template()

# Params
---

In [18]:
keyname_param = Parameter('keyName',
                          Description='existing key to use for ssh',
                          Type='String')
t.add_parameter(keyname_param);

# Maps
---

In [19]:
t.add_mapping('RegionMap', { 'us-east-1': {'ami': 'ami-b73b63a0'}
                           , 'us-east-2': {'ami': 'ami-58277d3d'}
                           , 'us-west-1': {'ami': 'ami-23e8a343'}
                           , 'us-west-2': {'ami': 'ami-5ec1673e'}
                           });

# Resources
---

## Network

In [20]:
vpc = ec2.VPC('myVpc', CidrBlock='10.0.0.0/16')
resources.add(vpc)

In [21]:
web_dmz = ec2.Subnet('webDmz', CidrBlock='10.0.0.0/24', VpcId=Ref(vpc))
resources.add(web_dmz)

In [22]:
igw = ec2.InternetGateway('igw')
attach_igw = ec2.VPCGatewayAttachment('attachIgw', VpcId=Ref(vpc), InternetGatewayId=Ref(igw))
resources.add(igw)
resources.add(attach_igw)

In [23]:
route_table = ec2.RouteTable('routeTable', VpcId=Ref(vpc))
gw_route = ec2.Route('gwRoute',
                     DestinationCidrBlock='0.0.0.0/0',
                     GatewayId=Ref(igw),
                     RouteTableId=Ref(route_table))
associate_route_table = ec2.SubnetRouteTableAssociation('associateRouteTable', 
                                                        RouteTableId=Ref(route_table),
                                                        SubnetId=Ref(web_dmz))
resources.add(route_table)
resources.add(gw_route)
resources.add(associate_route_table)

## Security Groups

In [24]:
allowed_ports = [22, 80, 443]
rules = [ec2.SecurityGroupRule(CidrIp='0.0.0.0/0', FromPort=p, ToPort=p, IpProtocol='tcp') for p in allowed_ports]

web_dmz_sg = ec2.SecurityGroup('webDmzSg',
                               DependsOn='attachIgw',
                               GroupDescription='allow http/s traffic to web nodes',
                               VpcId=Ref(vpc),
                               SecurityGroupIngress=rules)
resources.add(web_dmz_sg)

## Autoscaling

In [25]:
launch_config = LaunchConfiguration('launchConfig',
                                    DependsOn='attachIgw',
                                    AssociatePublicIpAddress=True,
                                    SecurityGroups=[Ref(web_dmz_sg)],
                                    ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                                    InstanceType='t2.micro',
                                    KeyName=Ref(keyname_param))

web_asg = AutoScalingGroup('webAsg',
                           VPCZoneIdentifier=[Ref(web_dmz)],
                           LaunchConfigurationName=Ref(launch_config),
                           AvailabilityZones=[GetAtt(web_dmz, 'AvailabilityZone')],
                           DesiredCapacity=1,
                           MinSize=1,
                           MaxSize=4)

resources.add(launch_config)
resources.add(web_asg)

# Outputs

# Build
---

In [26]:
for r in resources:
    t.add_resource(r)
    
with open('auto.json', 'w') as fd:
    fd.write(t.to_json())

In [27]:
!aws cloudformation validate-template --template-body file://auto.json > /dev/null && echo "ok" || echo "nope"

ok
